## 0. Dependency

In [1]:
import torch
import cv2 #opencv
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
device='cpu'

## 1. Python function-laplacian

In [ ]:
def laplacian(a):
    L=torch.zeros(6400).to(device)
    for i in range(6400):
        #NORTH BOUND
        if i<80:
            A_N=a[i]
        else:
            A_N=a[i-80]
        
        #SOUTH BOUND
        if i>=6320:
            A_s=a[i]
        else:
            A_S=a[i+80]
        
        #WEST BOUND
        if i%80==0:
            A_W=a[i]
        else:
            A_W=a[i-1]
        
        #EAST BOUND
        if i%80==79:
            A_E=a[i]
        else:
            A_E=a[i+1]
            
        L[i]=(A_E+A_W+A_N+A_S-4*a[i])*80*80
            
    return L

In [ ]:
def LAPLACIAN(A):
    size=A.size(dim=0)
    LP=torch.rand(size,6400).to(device)
    for i in range(size):
        LP[i]=laplacian(A[i]);
    return LP

## 2. Opencv-laplacian

In [22]:
def LAP(A, dim):
    LAP_A=torch.empty(len(A), dim, dim)
    ## Pre-process ########################################################################################################
    ### 1. Unflattened y_pred
    A=torch.nn.Sequential(torch.nn.Unflatten(1, (1,N,N)))(A)
    print("1. Unflattened y_pred, x.size(): ", A.size())
    print(A)
    
    ### 2. Zero-pad y-pred (e.g., 80*80->82*82)
    filters = torch.zeros(1, 1, 3, 3)
    filters[0][0][1][1]=1
    A=torch.nn.functional.conv2d(A, filters, padding=2)
    print("2. Zero-pad y-pred, x.size(): ", A.size())
    print(A)

    ### 3. Fill in BC into the zero-padded region (0-> first derivative=1)
    A=torch.transpose(A, 0,2)
    A[0]=A[1]
    A[N+1]=A[N]
    A=torch.transpose(A, 0,2)
    A=torch.transpose(A, 0,3)
    A[0]=A[1]
    A[N+1]=A[N]
    A=torch.transpose(A, 0,3)
    print("3. Fill in BC into the zero-padded region, x.size(): ", A.size())
    print(A)
    #######################################################################################################################
    ## Laplace ############################################################################################################
    ### 1. Tensor into numpy (Opencv takes numpy form) 
    A= A.numpy()
    #print("A.shape: ", A.shape)
    ### 2. Take laplacian with opencv
    for k in range(len(A)):
        temp=cv2.Laplacian(A[k][0], ddepth=-1, ksize=1)
        #print("temp.shape: ", temp.shape)
        temp=torch.tensor(temp)
        #print("temp.size(): ", temp.size())
        b=torch.transpose(temp[1:dim+1], 0,1)[1:dim+1] #(3, 3)
        #print("b: ", b.size())
        LAP_A[k]=b

    print("LAP_A: ")
    print(LAP_A)
    ### 3. Reform y_pred: flattened y_pred
    LAP_A=torch.flatten(LAP_A, start_dim=1)
    #######################################################################################################################
    return LAP_A

## Laplacian minimum test-2

In [24]:
## 0. An assumed y_pred
M=2
N=3
x=torch.rand(M, N*N)
print("0. An assumed y_pred, x.size(): ", x.size())
print(x)

x=LAP(x, dim=N)
print("4. Laplacian, x.size(): ", x.size())
print(x)

0. An assumed y_pred, x.size():  torch.Size([2, 9])
tensor([[0.3224, 0.4638, 0.7610, 0.3567, 0.1319, 0.0719, 0.2312, 0.5198, 0.7374],
        [0.4302, 0.0191, 0.9067, 0.4216, 0.0828, 0.5747, 0.7391, 0.4752, 0.9706]])
1. Unflattened y_pred, x.size():  torch.Size([2, 1, 3, 3])
tensor([[[[0.3224, 0.4638, 0.7610],
          [0.3567, 0.1319, 0.0719],
          [0.2312, 0.5198, 0.7374]]],


        [[[0.4302, 0.0191, 0.9067],
          [0.4216, 0.0828, 0.5747],
          [0.7391, 0.4752, 0.9706]]]])
2. Zero-pad y-pred, x.size():  torch.Size([2, 1, 5, 5])
tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.3224, 0.4638, 0.7610, 0.0000],
          [0.0000, 0.3567, 0.1319, 0.0719, 0.0000],
          [0.0000, 0.2312, 0.5198, 0.7374, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]],


        [[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.4302, 0.0191, 0.9067, 0.0000],
          [0.0000, 0.4216, 0.0828, 0.5747, 0.0000],
          [0.0000, 0.7391

## Laplacian minimum test-1

In [12]:
## 0. An assumed y_pred
M=10
N=80
x=torch.rand(M, N*N)
print("0. An assumed y_pred, x.size(): ", x.size())
#print(x)

## 1. Unflattened y_pred
x=torch.nn.Sequential(torch.nn.Unflatten(1, (1,N,N)))(x)
print("1. Unflattened y_pred, x.size(): ", x.size())
#print(x)

## 2. Zero-pad y-pred (e.g., 80*80->82*82)
filters = torch.zeros(1, 1, 3, 3)
filters[0][0][1][1]=1
x=torch.nn.functional.conv2d(x, filters, padding=2)
print("2. Zero-pad y-pred, x.size(): ", x.size())
#print(x)

## 3. Fill in BC into the zero-padded region (0-> first derivative=1)
x=torch.transpose(x, 0,2)
x[0]=x[1]
x[N+1]=x[N]
x=torch.transpose(x, 0,2)
x=torch.transpose(x, 0,3)
x[0]=x[1]
x[N+1]=x[N]
x=torch.transpose(x, 0,3)
print("3. Fill in BC into the zero-padded region, x.size(): ", x.size())
#print(x)

## 4. Laplacian
x=LAP(x, dim=N)
print("4. Laplacian, x.size(): ", x.size())
#print(x)

0. An assumed y_pred, x.size():  torch.Size([10, 6400])
1. Unflattened y_pred, x.size():  torch.Size([10, 1, 80, 80])
2. Zero-pad y-pred, x.size():  torch.Size([10, 1, 82, 82])
3. Fill in BC into the zero-padded region, x.size():  torch.Size([10, 1, 82, 82])
4. Laplacian, x.size():  torch.Size([10, 6400])


## Laplacian BC, minimum test

In [176]:
x=torch.ones(2, 1, 3, 3)
filters = torch.zeros(1, 1, 3, 3)
filters[0][0][1][1]=1
y=torch.nn.functional.conv2d(x, filters, padding=2)
print(y.size())
b=LAP(y, ks=1, dim=5)
print(b.size())

torch.Size([2, 1, 5, 5])
A.shape:  (2, 1, 5, 5)
torch.Size([2, 1, 9])


## Laplacian BC, minimum test

In [135]:
print(y[1].size())
b=LAP(y[1], ks=1, dim=5)
print(b.size())

torch.Size([1, 5, 5])
A.shape:  (1, 5, 5)
temp.shape:  (5, 5)
temp.size():  torch.Size([5, 5])
b:  torch.Size([3, 3])
LAP_A: 
tensor([[[-2., -1., -2.],
         [-1.,  0., -1.],
         [-2., -1., -2.]]])
torch.Size([1, 9])


In [140]:
temp=torch.ones(1, 5, 5)
templ=torch.ones(1, 3, 3)
print(temp.size())
b=temp[0][1:4]
print(b.size())
b=torch.transpose(b, 0,1)
print(b.size())
templ[0]=b[1:4]
print(templ[0].size())
#b=torch.transpose(temp[1:4], 0,1)[1:4]

torch.Size([1, 5, 5])
torch.Size([3, 5])
torch.Size([5, 3])
torch.Size([3, 3])


In [154]:
a=torch.ones(1, 5, 5) #(1, 5, 5)
print(a.size())
b=torch.transpose(a[0][1:4], 0,1)[1:4] #(3, 3)
print(b.size())
a=torch.ones(1, 3, 3) 
a[0]=b
print(a.size())

torch.Size([1, 5, 5])
torch.Size([3, 3])
torch.Size([1, 3, 3])


## Laplacian correctness, minimum test

In [268]:
d=5
x=torch.empty(2,d*d)
for i in range(d*d):
    x[0][i]=i+1
    x[1][i]=i+1
x=torch.rand(2,d*d)

x[0][4]=1
x[0][7]=1.123
x[0][13]=1

x=torch.nn.Sequential(torch.nn.Unflatten(1, (1,d,d)))(x)
print(x)
print(x.size())
LAP(x, ks=1, dim=3)

tensor([[[[0.3992, 0.8774, 0.6634, 0.2342, 1.0000],
          [0.1947, 0.4061, 1.1230, 0.8821, 0.9763],
          [0.3167, 0.4991, 0.3160, 1.0000, 0.6228],
          [0.2018, 0.3346, 0.3182, 0.3023, 0.6560],
          [0.5153, 0.2915, 0.3006, 0.7925, 0.2103]]],


        [[[0.7742, 0.4156, 0.8498, 0.4729, 0.9797],
          [0.3220, 0.2945, 0.4374, 0.1736, 0.0972],
          [0.2329, 0.8130, 0.5133, 0.0437, 0.6964],
          [0.5253, 0.3982, 0.2505, 0.4501, 0.5961],
          [0.9192, 0.5869, 0.4556, 0.7558, 0.8553]]]])
torch.Size([2, 1, 5, 5])
A.shape:  (2, 1, 5, 5)
temp.shape:  (5, 5)
temp.size():  torch.Size([5, 5])
b:  torch.Size([3, 3])
temp.shape:  (5, 5)
temp.size():  torch.Size([5, 5])
b:  torch.Size([3, 3])
LAP_A: 
tensor([[[[ 1.0698, -0.6231, -0.0279],
          [-2.2243,  1.6762, -0.0190],
          [-0.1950, -1.8767,  1.5573]]],


        [[[ 0.8102, -1.8132,  0.5829],
          [ 0.0814, -0.5084,  0.8150],
          [ 0.3569,  1.6585, -0.1542]]]])


tensor([[[ 1.0698, -0.6231, -0.0279, -2.2243,  1.6762, -0.0190, -0.1950,
          -1.8767,  1.5573]],

        [[ 0.8102, -1.8132,  0.5829,  0.0814, -0.5084,  0.8150,  0.3569,
           1.6585, -0.1542]]])

In [269]:
M=2
N=3
x=torch.rand(M, N*N)
print(x.size())
x=torch.nn.Sequential(torch.nn.Unflatten(1, (1,N,N)))(x)
print(x.size())

## Gernate larger
filters = torch.zeros(1, 1, 3, 3)
filters[0][0][1][1]=1
x=torch.nn.functional.conv2d(x, filters, padding=2)
#print(x)

## Fill in BC
x=torch.transpose(x, 0,2)
x[0]=x[1]
x[N+1]=x[N]
x=torch.transpose(x, 0,2)
x=torch.transpose(x, 0,3)
x[0]=x[1]
x[N+1]=x[N]
x=torch.transpose(x, 0,3)
print(x)

#Laplacian
x=LAP(x, ks=1, dim=N)
print(x)

torch.Size([2, 9])
torch.Size([2, 1, 3, 3])
tensor([[[[0.8597, 0.8597, 0.1897, 0.3004, 0.3004],
          [0.8597, 0.8597, 0.1897, 0.3004, 0.3004],
          [0.3930, 0.3930, 0.6542, 0.2018, 0.2018],
          [0.7750, 0.7750, 0.7426, 0.1945, 0.1945],
          [0.7750, 0.7750, 0.7426, 0.1945, 0.1945]]],


        [[[0.5845, 0.5845, 0.6952, 0.6445, 0.6445],
          [0.5845, 0.5845, 0.6952, 0.6445, 0.6445],
          [0.5378, 0.5378, 0.2068, 0.6407, 0.6407],
          [0.3451, 0.3451, 0.3538, 0.1235, 0.1235],
          [0.3451, 0.3451, 0.3538, 0.1235, 0.1235]]]])
A.shape:  (2, 1, 5, 5)
temp.shape:  (5, 5)
temp.size():  torch.Size([5, 5])
b:  torch.Size([3, 3])
temp.shape:  (5, 5)
temp.size():  torch.Size([5, 5])
b:  torch.Size([3, 3])
LAP_A: 
tensor([[[[-1.1367,  1.1100, -0.4145],
          [ 1.2452, -1.0895, -0.6041],
          [-0.2094,  0.5438,  0.5554]]],


        [[[ 0.0641, -0.4770,  0.2014],
          [-0.6499,  1.4002, -0.3859],
          [ 0.0469, -0.9472,  0.7474]]]])
tenso

## Copy array

In [93]:
a=torch.ones(1, 5, 5) #(1, 5, 5)
print(a.size())
b=torch.transpose(a[0][1:4], 0,1)[1:4] #(3, 3)
print(b.size())
a=torch.ones(1, 3, 3) 
a[0]=b
print(a.size())

torch.Size([1, 5, 5])
torch.Size([3, 3])
torch.Size([1, 3, 3])


## GPU-numpy test

In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import cv2
device=torch.device('cuda:0')

In [19]:
x=torch.randn(10, 6400)
x=x.to(device)
print(x.size())

torch.Size([10, 6400])


In [31]:
x=torch.randn(1, 5).to(device)
LAP_A=torch.empty(1, 5).to(device)
print(type(x), x)
x= torch.Tensor.cpu(x).numpy()
print(type(x), x)
x=torch.tensor(x,requires_grad=True)
print(type(x), x)
LAP_A=x.to(device)
print(type(LAP_A), LAP_A)

<class 'torch.Tensor'> tensor([[-0.6532,  1.0943,  1.0201,  0.0942,  1.5916]], device='cuda:0')
<class 'numpy.ndarray'> [[-0.6531749   1.0942525   1.020143    0.09419392  1.5915849 ]]
<class 'torch.Tensor'> tensor([[-0.6532,  1.0943,  1.0201,  0.0942,  1.5916]], requires_grad=True)
<class 'torch.Tensor'> tensor([[-0.6532,  1.0943,  1.0201,  0.0942,  1.5916]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)


## Unknown

In [5]:
x=torch.ones(100,6400)
x=x.to(device)
print(x.size())
print(x)

torch.Size([100, 6400])
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


In [19]:
xn= torch.nn.Sequential(torch.nn.Unflatten(1, (1,80,80)))(x).numpy()
print(len(xn))

100


In [29]:
yn=xn
for i in range(len(xn)):
    yn[i]=cv2.Laplacian(xn[i], ddepth=-1)

In [30]:
yn=torch.tensor(yn)
yn=torch.flatten(yn, start_dim=1)

In [31]:
print(yn.size())

torch.Size([100, 6400])


In [3]:
for i in x:
    i_f= torch.nn.Sequential(torch.nn.Unflatten((1,80,80)))(i)
    print(i_f.size())

TypeError: __init__() missing 1 required positional argument: 'unflattened_size'

In [43]:
print("hello world")
pppprint("hello world") #syntax error

hello world


NameError: name 'pppprint' is not defined

In [ ]:
def flaplacian(a):
    a=5*a
          
    return a

In [ ]:
def FLAPLACIAN(A):
    size=A.size(dim=0)
    LP=torch.rand(size,6400).to(device)
    for i in range(size):
        LP[i]=flaplacian(A[i]);
    return LP

In [15]:
LAP(x)

A:  [[[[ 1.  2.  3.  4.  5.]
   [ 6.  7.  8.  9. 10.]
   [11. 12.  0. 14. 15.]
   [16. 17. 18. 19. 20.]
   [21. 22. 23. 24. 25.]]]]
len(A):  1
len(B[i]):  [[[ 10.  10.  10.  10.  10.]
  [  0.   0. -13.   0.   0.]
  [  0.   0.  26.   0.   0.]
  [  0.   0. -13.   0.   0.]
  [-10. -10. -10. -10. -10.]]]
len(B):  [[[[ 10.  10.  10.  10.  10.]
   [  0.   0. -13.   0.   0.]
   [  0.   0.  26.   0.   0.]
   [  0.   0. -13.   0.   0.]
   [-10. -10. -10. -10. -10.]]]]


tensor([[ 10.,  10.,  10.,  10.,  10.,   0.,   0., -13.,   0.,   0.,   0.,   0.,
          26.,   0.,   0.,   0.,   0., -13.,   0.,   0., -10., -10., -10., -10.,
         -10.]])

In [14]:
print(xn.size())

NameError: name 'xn' is not defined

In [2]:
x=torch.ones(100,25)
x=x.to(device)

## preserved

In [ ]:
    A_1=np.empty([1, dim+2, dim+2])
    for k in range(len(A)):
        for i in range(dim+2):
            for j in range(dim+2):
                if i==0:
                    A_1[i][j]=A[k][i][j-1]
                elif i==dim+1:
                    A_1[i][j]=A[k][i-2][j-1]
                elif j==0:
                    A_1[i][j]=A[k][i-1][j]
                elif j==dim+1:
                    A_1[i][j]=A[k][i-1][j-2]
                else:
                    A_1[i][j]=A[k][i-1][j-1]
    print("A_1: ")
    print(A_1)
    print(A_1.shape)